In [1]:
# import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
import os
# import cv2

import Utils

In [2]:
image_data = Utils.read_data("Berea_2d25um_binary.raw")
image_data.shape


(1000, 1000, 1000)

In [3]:
voxels = Utils.extract_subvolumes(image_data)
voxels.shape

(64, 250, 256, 256, 1)

In [4]:
import numpy as np
from skimage import morphology, measure

In [10]:


# Load the binary porous image
image = voxels[0][4]  # Replace with your binary image data

# Skeletonize the image
skeleton = morphology.skeletonize(image)

# Calculate the lineal-path function
lineal_path = np.sum(skeleton)/np.sum(image)

# Print the lineal-path value
print(lineal_path)

14.232801762772633


In [11]:
# image = ...  # Replace with your binary image data

# # Calculate the two-point correlation function
# correlation = morphology.correlation_2d(image)

# # Print the correlation matrix
# print(correlation)

AttributeError: module 'skimage.morphology' has no attribute 'correlation_2d'

In [13]:
# Load the binary porous image
image = image_data[0]  # Replace with your binary image data

# Label the connected components
labels = measure.label(image, connectivity=1)

# Calculate the two-point cluster function
unique_labels, label_counts = np.unique(labels, return_counts=True)
cluster_function = np.sum(label_counts[label_counts > 1]) / np.sum(label_counts)

# Print the cluster function
print(cluster_function)

0.999994


In [2]:
import porespy

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject